In [1]:
#import library
import requests
import pandas as pd
from datetime import datetime
import pickle
import numpy as np
import pickle

from finta import TA
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
import os
import simdkalman
from sklearn.ensemble import IsolationForest
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from tqdm import notebook
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import warnings
warnings.simplefilter('ignore')

In [2]:
def get_symbol(best_model_tuning):
    symbols = {}
    list_sym = []
    for sym in best_model_tuning.keys():
        list_sym.append(sym)
    symbols['all'] = list_sym
    return symbols

In [3]:
def get_keysymbol(best_model_tuning):
    key_symbols = {}
    for sym in best_model_tuning.keys():
        best_cv = list(best_model_tuning[sym].keys())
        key_symbols[sym] = best_cv
    return key_symbols

In [4]:
def load_sym_model():
    with open ('best_model_tuning.pkl','rb') as output:
        best_model_tuning = pickle.load(output)
    key_symbols = get_keysymbol(best_model_tuning)
    return best_model_tuning,key_symbols

In [5]:
def get_data(symbols: list, timeframe: str = "D1"):
    start = "2018-01-01"
    end = datetime.now().date().isoformat()
    data_new = {}
    params = {
            "timeframe": timeframe,
            "start": start,
            "end": end,
        }
    for sym in symbols['all']:
        print(sym)
        response = requests.get(f"http://221.132.33.180:8005/history/{sym}", params=params)
        response.raise_for_status()
        data = response.json()
        data = pd.DataFrame(data)
        data["Date"] = pd.to_datetime(data['Date'], unit='s')
        data.set_index("Date", inplace=True)
        data_new[sym] = data
    return data_new

In [6]:
def create_indicators(ohlcv: pd.DataFrame) -> pd.DataFrame:
    data = ohlcv.copy()
    indi, signal = bias(ohlcv)
    data = pd.concat([data, indi], axis=1)
    data['BIAS_signal'] = signal
    data['VR'], data["VR_signal"] = vr(ohlcv)
    data['TRIX'], data["TRIX_signal"] = trix(ohlcv)
    data['ER'] = TA.ER(ohlcv)
    data['EVWMA'] = TA.EVWMA(ohlcv)
    data['VWAP'] = TA.VWAP(ohlcv)
    data['MOM'] = TA.MOM(ohlcv)
    data['ROC'] = TA.ROC(ohlcv)
    data['RSI'] = TA.RSI(ohlcv)
    data['IFT_RSI'] = TA.IFT_RSI(ohlcv)
    data['ATR'] = TA.ATR(ohlcv)
    data['BBWIDTH'] = TA.BBWIDTH(ohlcv)
    data['ADX'] = TA.ADX(ohlcv)
    data['STOCH'] = TA.STOCH(ohlcv)
    data['STOCHD'] = TA.STOCHD(ohlcv)
    data['AO'] = TA.AO(ohlcv)
    data['MI'] = TA.MI(ohlcv)
    data['MFI'] = TA.MFI(ohlcv)
    data['PZO'] = TA.PZO(ohlcv)
    data['EFI'] = TA.EFI(ohlcv)
    data['EMV'] = TA.EMV(ohlcv)
    data['CCI'] = TA.CCI(ohlcv)
    data['FISH'] = TA.FISH(ohlcv)
    data['FVE'] = TA.FVE(ohlcv)

    macd = TA.MACD(ohlcv)
    data['MACDCal'] = macd['MACD'] - macd['SIGNAL']

    macdev = TA.EV_MACD(ohlcv)
    data['EVMACD'] = macdev["MACD"]

    data['TR'] = TA.TR(ohlcv)

    DMI = TA.DMI(ohlcv)
    data['DMI+'] = DMI["DI+"]
    data['DMI-'] = DMI["DI-"]

    VORTEX = TA.VORTEX(ohlcv)
    data['VIp'] = VORTEX["VIp"]
    data['ADL'] = TA.ADL(data)

    TSI = TA.TSI(ohlcv)
    data['TSI'] = TSI["TSI"]
    data['TSIsignal'] = TSI["signal"]

    KST = TA.KST(ohlcv)
    data['KST'] = KST["KST"]

    data['CHAIKIN'] = TA.CHAIKIN(ohlcv)
    data['OBV'] = TA.OBV(ohlcv)
    data['WOBV'] = TA.WOBV(ohlcv)

    EBBP = TA.EBBP(ohlcv)
    data['EBBPBull'] = EBBP["Bull."]
    data['EBBPBear'] = EBBP["Bear."]

    BASPN = TA.BASPN(ohlcv)
    data['BASPNBuy'] = BASPN["Buy."]
    data['BASPNSell'] = BASPN["Sell."]
    data['COPP'] = TA.COPP(ohlcv)

    BASP = TA.BASP(ohlcv)
    data['BASPBuy'] = BASP["Buy."]
    data['BASPSell'] = BASP["Sell."]

    WTO = TA.WTO(ohlcv)
    data['WTOWT1'] = WTO["WT1."]

    data['STC'] = TA.STC(ohlcv)
    data['VPT'] = TA.VPT(ohlcv)
    return data

LOOK_BACK = 10
"""Hàm này dùng để phân chia data ra theo từng ngày nối đuôi nhau vựa trên lock_back"""
# Xem xem trung bình 10 ngày tăng hay giảm, là dùng để xem xu hướng của tiền
def create_dataset(data, label, look_back=1):  #
    X_ = []
    y_ = []
    # Tạo vòng lặp để lấy các lookback
    for i in range(len(data)-look_back-1):
        X_.append(data[i:(i+look_back)])
        y_.append(label[i + look_back])
    return np.array(X_), np.array(y_)

""" Hàm này để lấy các ngày liên tiếp theo n_days để làm các cột dữ liệu"""
def add_past_days_as_feature(data: pd.DataFrame, n_days: int = 5):
    data = pd.concat([data.shift(i).add_suffix(f"_{i}") for i in range(n_days)], axis=1)
    return data

" Hàm Bias dùng để so sánh giữ hai hai tính hiệu dài hạn và ngắn hạn vựa trên short_val và long_val "
def bias(prices):
    short_avg = prices['Close'].rolling(3, min_periods=1).mean()
    long_avg = prices['Close'].rolling(5, min_periods=1).mean()

    short_val = pd.Series(((prices['Close'] - short_avg) / short_avg) * 100, name="BIAS_short", index=prices.index)
    long_val = pd.Series(((prices['Close'] - long_avg) / long_avg) * 100, name="BIAS_long", index=prices.index)
    indi = pd.concat([short_val, long_val], axis=1)

    # So sánh xem dài hạn hay ngắn hạn cái nào sẽ lời hơn
    signal = pd.Series((long_val > short_val).astype(int), name="BIAS_signal", index=prices.index)
    return indi, signal

def vr(prices):
    maximum = (prices['High'] + prices['Close'].shift(1).bfill()).mean() # Lấy giá cao nhất ngày hiện tạo cộng cho ngày đóng cửa của tương lai,
    # Nếu tương lai là Nan thì cộng cho giá đóng cưa hiện tại
    minimum = (prices['Low'] + prices['Close'].shift(1).bfill()).mean()
    high = prices['High'].rolling(14, min_periods=1).mean()
    low = prices['Low'].rolling(14, min_periods=1).mean()

    # Tính chỉ số
    indi = pd.Series((maximum - minimum) / (high - low), name="VR", index=prices.index)
    signal = pd.Series((indi > 0.5).astype(int), name="VR_signal", index=prices.index)
    return indi, signal

# Hàm tính giá trị Trix
def trix(prices):
    indi = TA.TRIX(prices, 10)
    signal = pd.Series((indi < 0).astype(int), name="TRIX_signal", index=prices.index)
    return indi, signal

In [7]:
def set_up_features(data_new):
    price = {}
    # Lấy data của giá đóng cửa của loại đồng tiền
    for symbol in data_new.keys():
        price[symbol] = data_new[symbol]['Close'].copy()
    y = {}
    # y ở đây là ngày mai tăng hay giảm
    # Y lấy dữ liệu giá đóng cửa. Nếu giá trị đang Nan thì điền vào giá trị gần nhất của cột Close
    for symbol in data_new.keys():
        #y[symbol] = data_new[symbol]["Close"].shift(-1).ffill()
        y[symbol] = data_new[symbol]["Close"].ffill()
    # x là features
    X = {}
    # Đoạn này lấy dữ liệu X là data dùng để tách train, test
    for symbol in data_new.keys():
        X_base_features = create_indicators(data_new[symbol])
        X[symbol] = add_past_days_as_feature(data=X_base_features, n_days=LOOK_BACK)
        X[symbol]["label"] = y[symbol].copy()
        X[symbol] = X[symbol].dropna(axis=0)
        y[symbol] = X[symbol]["label"].copy()
    return X,y

In [8]:
def split_data_scale(X: dict, y: dict, symbols: dict):
    data_all_money = {}
    for symbol in symbols["all"]:
        # Lấy dữ liệu gốc khi (chưa scale)
        data_original = X[symbol].copy()
        y_ = pd.Series(y[symbol])
        X_train = data_original.iloc[:-1]
        y_train =y_.iloc[:-1]
        X_test = data_original.iloc[-1]
        y_test =y_.iloc[-1]
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        data_all_money[symbol] = [X_train_scaled, X_test, y_train, y_test]
    return data_all_money

In [9]:
#  Ta đi smooth data để smooth ta cần biết các dữ liệu nhiễu thường xuyên, và bộ dữ liệu thực tế
# Việc cần ở đây là chỉ số nhiễu tượng trưng cho dữ liệu đó chỉ cần chỉnh bao nhiêu phần trăm bị nhiễu
def Smoothing_data(y, symbols) -> dict:
  symbolsmooths = {}
  # cho nhiều bộ smooths y khác nhau để xem cái nào tốt nhất
  for syms in symbols["all"]:
    smooths = []
    for isf in range(4,5):
        for ins in range(4,5):
            smoothing_factor = isf
            n_seasons = ins
            # --- define state transition matrix A tạo ma trận n_season+1
            state_transition = np.zeros((n_seasons+1, n_seasons+1))
            # hidden level         Đặt mặc định số đầu tiên là 1
            state_transition[0,0] = 1
            # season cycle         Sau đó ta chỉnh các thông sô khác cho mặc định dòng 1 có n_season = -1
            state_transition[1,1:-1] = [-1.0] * (n_seasons-1)
            # Tạo đường chéo chính có giá trị là 1
            state_transition[2:,1:-1] = np.eye(n_seasons-1)
            # --- observation model H
            # observation is hidden level + weekly seasonal compoenent
            observation_model = [[1,1] + [0]*(n_seasons-1)]
            # --- noise models, parametrized by the smoothing factor
            level_noise = 0.2 / smoothing_factor
            observation_noise = 0.2
            season_noise = 1e-3
            process_noise_cov = np.diag([level_noise, season_noise] + [0]*(n_seasons-1))**2
            observation_noise_cov = observation_noise**2
            # Sử dụng Kalman filter
            kf = simdkalman.KalmanFilter(
                state_transition, # Hiển thị cách model chuyển tiếp để thay đổi ( Đây là cách nhìn dữ liệu bị nhiễu)
                process_noise_cov, # ma trận biểu thị mức độ nhiễu của mô hình (Đây là để biết nhiễu thường xảy ra với tuần suât nào)
                observation_model, #  Là mô hình quan sát của hệ thông thường là ma trận hay hàm số biểu thị (Đây là  kết quả hiện tại đang cần chỉnh nhiễu)
                observation_noise_cov #  Ma trận biểu thị mức độ nhiễu trong đo lường  (Đây giống như là sai số để xem kết quả dự đoán với thực tế thì sau bnhieu)
                )
            # Sau đó ta tính toán lấy ra được các dự đoán nhiễu và trã về dự đoán bị nhiễu được quy định bằng các tham số trên
            block = y[syms]
            n_train = block.shape[0]
            n_test = 60
            result = kf.compute(block, n_test)
            predictproba = result.smoothed.states.mean[:,0]
            y_label = []
            for ivalue in range(1,len(predictproba)):
                if(predictproba[ivalue] > predictproba[ivalue-1]):
                    y_label.append(1)
                else:
                    y_label.append(-1)
            smooths.append(y_label)
    symbolsmooths[syms] = smooths
  return symbolsmooths

In [10]:
def smooth_label(y_train_ ,symbolsmooths, syms ):
  # Hàm chỉnh dữ liệu lại cho smooth
  y_train = []
  y_test = []
  # chạy từng mã gán lại nhãn đã smooth, -1 là 0 giảm, 1 là 1 tăng
  for idx in range(len(y_train_)):
      if symbolsmooths[syms][0][idx]  == -1:
          y_train.append(0)
      else:
          y_train.append(1)
  # y_test cũng vậy, phần test sẽ là phần còn lại tính từ n_split
  for idx in range(len(symbolsmooths[syms][0]) - len(y_train_)):
      if symbolsmooths[syms][0][idx+ len(y_train_)] == -1:
          y_test.append(0)
      else:
          y_test.append(1)
  y_test.append(1)
  return y_train, y_test

In [11]:
def create_train_test_all_data_forex(X, y, symbols):
  data_all = {}
  data_all_money = split_data_scale(X,y, symbols)
  symbolsmooth_y = Smoothing_data(y, symbols)

  for sym in symbols["all"]:
    # Lấy các cột của dữ liệu
    df = pd.DataFrame(X[sym] )
    col_df = df.columns

    # Lấy đúng loại tiền cần dùng
    X_train, X_test, y_train, y_test = data_all_money[sym]
    # Sau khi smooth data ta có y_train và y_test mới
    y_train, y_test = smooth_label( y_train, symbolsmooth_y, sym)
    # Sau đó  tạo ra các dataframe mới
    X_train = pd.DataFrame(X_train, columns = col_df)
    X_test =  X_test.to_frame().T
    # Drop các cột không cần thiết là label đi
    X_train = X_train.drop(columns = ["label"])
    X_test = X_test.drop(columns = ["label"])
    # Lấy dữ liệu tiền tệ đó ra
    data_set = pd.get_dummies(X_train, drop_first=False)
    data_all[sym] = [data_set, X_test, y_train, y_test]
  return data_all

In [12]:
# Hàm khởi tạo Isolation forest cho việc loại bỏ các outlier vựa trên decision Tree
# Nó chỉ loại bỏ 1 phần bị outliers
def create_clf(X_train):
  clf = IsolationForest(
    n_estimators=100,
    max_samples='auto',
    n_jobs=-1,
    random_state=2023)
  clf.fit(X_train)
  return clf
# hàm tạo ngưỡng nhiễu
def outlier_threshold(normality, k=1.5):
  q1 = np.quantile(normality, 0.2)
  q3 = np.quantile(normality, 0.8)
  threshold = q1 - k*(q3-q1)
  return threshold
# Hàm loại bỏ outlier cho tập data train
def delete_outlier(clf, X_train, y_train):
  # Lấy ngưỡng cho dữ liệu
  normality_df = pd.DataFrame(clf.decision_function(X_train), columns=['normality'])
  threshold = outlier_threshold(normality_df['normality'].values, k=1.5)
  # Sau đo ta loại bỏ dòng bị outlier
  X_train = X_train[normality_df['normality'].values>=threshold]
  y_train = y_train[normality_df['normality'].values>=threshold]
  return X_train, y_train

def data_np(data_all, symbols):
  for sym in symbols["all"]:
    # Thay đổi dữ liệu về dạng làm tròn và numpy
    data_all[sym][2] = np.array(data_all[sym][2])
    data_all[sym][3] = np.array(data_all[sym][3])
    data_all[sym][0] = data_all[sym][0].astype(np.float32)
    data_all[sym][1] = data_all[sym][1].astype(np.float32)
  return data_all

def delete_outlier_data(data_all, symbols):
  data_all = data_np(data_all, symbols)
  for sym in symbols["all"]:
    clf = create_clf(data_all[sym][0])
    data_all[sym][0], data_all[sym][2] = delete_outlier(clf, data_all[sym][0], data_all[sym][2])
  return data_all

In [13]:
def check_missing_symbols(symbols,key_symbols):
    mmissing_symbols = []
    for sym in symbols["all"]:
        if sym  not in key_symbols.keys():
            mmissing_symbols.append(sym)
    return mmissing_symbols

In [14]:
def get_best_cv_for_miss_symbols(missing_symbols):
  with open('best-cv-rf-and-xgb-and-mlp.pkl', "rb") as file:
    best_cv_df_new= pickle.load(file)
    best_cv_df ={}
  for sym in missing_symbols:
    sorted_df = best_cv_df_new[sym].copy().sort_values(by=['model', 'accuracy'], ascending=[True, False])
    best_cv_df[sym] = sorted_df
  for sym in missing_symbols:
    best_cv_df[sym] = pd.DataFrame(best_cv_df[sym])
  return best_cv_df

In [15]:
# hàm này để đánh giá mô hình tổng
# Là khi chúng ta train ra hết 3 model chúng ta xem rằng chúng kết hợp với nhau thì kết quả cuối cùng độ chính xác là bao nhiêu
def CV_ensemble(ensemble_name, ensemble_func, estimators, X_train, y_train, n_folds=5, shuffle=True, random_state=2023):
  kf = KFold(n_splits=5, random_state=random_state, shuffle=True) # Tạo số lượng dữ liệu bằng Kfold sau đó shuffle

  res_list = [] # tạo res_list lưu thông tin của từng lần chia kfold
  # Chạy tập dữ liệu Kfold
  for train_idx, valid_idx in notebook.tqdm(kf.split(X_train), total=kf.get_n_splits(), desc='Eval_CV'): # Chạy theo số lần n_splits
    # print(len(X_train))
    # print(len(y_train))
    X_train_train, X_valid = X_train[train_idx], X_train[valid_idx]
    y_train_train, y_valid = y_train[train_idx], y_train[valid_idx]
    # Tổng hợp kết quả test đối vơi từng cặp k riêng
    ensemble_pred_proba = ensemble_func(estimators, X_train_train, y_train_train, X_valid)
    # Lấy các đánh giá  của model với tập train test k này
    neg_log_loss = np.negative(log_loss(y_valid, ensemble_pred_proba))
    accuracy = accuracy_score(y_valid, ensemble_pred_proba.argmax(axis=1))

    res_list.append([ensemble_name, neg_log_loss, accuracy]) # Lưu kết quả lại`
  res_df = pd.DataFrame(np.vstack((res_list)))
  res_df.columns = ['model', 'log_loss', 'accuracy']  # thêm các thông số để đánh giá
  return res_df

# hàm này để dự đoán bằng cách tổng hợp 3 model có train lại
def ensemble_average(estimators, X_train, y_train, X_test):
  # Hàm này vựa trên việc lấy xác suất của các model vote cái nào có xác suất cao nhất thì chọn
  preds = []
  num_estimators = len(estimators)
  num_class = len(np.unique(y_train))
  for iter in range(num_estimators):
    y_train = np.array(y_train, dtype=np.int64)
    estimators[iter].fit(X_train, y_train)
    preds.append(estimators[iter].predict_proba(X_test))

  preds_stack = np.hstack((preds))
  preds_mean = []
  for iter in range(num_class):
    col_idx = np.arange(iter, num_estimators * num_class, num_class)
    preds_mean.append(np.mean(preds_stack[:,col_idx], axis=1))

  avg_pred = np.vstack((preds_mean)).transpose()
  return avg_pred

# hàm này đưa ra dự đoán mà ko cần train lại
def ensemble_average_model(estimators, X_test,y_train): # esitamtors là các model và X_test là dữ liệu train
  preds = []
  num_estimators = len(estimators)
  num_class = len(np.unique(y_train))
  # đoạn này lấy tổng dữ đoán của các model trong estimators
  for iter in range(num_estimators):
    preds.append(estimators[iter].predict_proba(X_test))
  preds_stack = np.hstack((preds))
  preds_mean = []
  # Dự đoán trung bình cho mỗi nhãn
  for iter in range(num_class):
    col_idx = np.arange(iter, num_estimators * num_class, num_class) # Tính cho số cột cho nhãn được dự đoán
    preds_mean.append(np.mean(preds_stack[:,col_idx], axis=1)) # Tính giá trị trung bình cho dự đoán

  avg_pred = np.vstack((preds_mean)).transpose()# Lấy hết các giá trị tring bình sau đó chuyển vị
  return avg_pred

In [16]:
def train_model(data_all,missing_symbols,best_cv_df):
      res_df_all = {}
      estimators_all_sym = {}
      for sym in missing_symbols:
      # Lưu trữ sym vét cạn
            estimators_all ={}
            X_train = data_all[sym][0]
            y_train = data_all[sym][2]
            X = np.array(X_train)
            y = np.array(y_train, dtype=np.int64)
            # load 3 cấu hình tốt nhất cho 3 model vào biến rf xbg và mlp
            rf1 = RandomForestClassifier(**eval(best_cv_df[sym].loc[best_cv_df[sym]['model']=='rf', 'best_hyper_param'].values[0]))
            xgb1 = XGBClassifier(**eval(best_cv_df[sym].loc[best_cv_df[sym]['model']=='xgb', 'best_hyper_param'].values[0]))
            mlp1 = MLPClassifier(**eval(best_cv_df[sym].loc[best_cv_df[sym]['model']=='mlp', 'best_hyper_param'].values[0]))
            # quá trình tổng hopkw 3 model để tạo ra model trung bình tốt nhất
            estimators = [rf1, xgb1, mlp1]
            estimators_name = 'rf_xgb_mlp'
            ensemble_name = 'average' + '_by_' + estimators_name + str(0) + str(0) + str(0)
            # Lấy res_df ra
            res_df = CV_ensemble(ensemble_name, ensemble_average, estimators, X, y, n_folds=5, shuffle=True, random_state=2023)
            sym_brute_force = sym + str(0) + str(0) + str(0)
            res_df_all[sym_brute_force] = res_df

            estimators_all[sym_brute_force] = estimators
            print(sym_brute_force)
            print(res_df)
            estimators_all_sym[sym] = estimators_all
      return estimators_all_sym

In [17]:
def save_model(best_model_tuning,best_model_missing):
    for sym in best_model_missing.keys():
        best_model_tuning[sym] =  best_model_missing[sym]
    file_name = 'best-cv-rf-and-xgb-and-mlp.pkl.pkl'
    # Lưu từ điển vào tệp pickle
    with open(file_name, 'wb') as file:
        pickle.dump(best_model_tuning, file)
    return best_model_tuning

In [18]:
def get_X_test_y_train(data_all,symbols):
    X_test = {}
    for symbol in symbols['all']:
        X_test[symbol] = data_all[symbol][1].copy()
    y_train = data_all[symbol][2]
    return X_test,y_train

In [19]:
def predict_sym(estimators_sym,key_symbols,X_test,y_train):
  pred_y_label = {}
  for sym in key_symbols:
    estimators = estimators_sym[sym]
    pred_y = ensemble_average_model(estimators,X_test,y_train)
    pred_y = pred_y.argmax(axis=1)
    pred_y_label[sym] = pred_y
  return pred_y_label

In [20]:
def predict_all_sym(estimators_all_sym,data_all):
  symbols = get_symbol(estimators_all_sym)
  key_symbols = get_keysymbol(estimators_all_sym)
  X_test,y_train = get_X_test_y_train(data_all,symbols)
  pred_y_label_all ={}
  for sym in symbols['all']:
    pred_y_label = predict_sym(estimators_all_sym[sym],key_symbols[sym],X_test[sym],y_train)
    pred_y_label_all[sym] = pred_y_label
  return pred_y_label_all  


In [21]:
def simplify_output(data):
  simplified = {}
  # Duyệt qua từng cặp tiền tệ
  for pair, signals in data.items(): 

    # Đếm tín hiệu mua/bán
    signal_counts = {'buy': 0, 'sell': 0}
    
    for name, signal in signals.items():
      if signal[0] == 1:
        signal_counts['buy'] += 1  
      else:
        signal_counts['sell'] += 1

    # Nếu toàn bộ là mua hoặc bán
    if signal_counts['buy'] == 0 or signal_counts['sell'] == 0:  
      simplified[pair] = 1 if signal_counts['buy'] > 0 else 0
    # Nếu có cả mua và bán    
    else:
      if signal_counts['buy'] > signal_counts['sell']:
        simplified[pair] = 1
      else: 
        simplified[pair] = 0
        
  return simplified

In [22]:
def get_signal(symbols):
    best_model_tuning,key_symbols = load_sym_model()
    missing_symbols = check_missing_symbols(symbols,key_symbols)
    data_new = get_data(symbols)
    X,y = set_up_features(data_new)
    data_all = create_train_test_all_data_forex(X,y,symbols)
    data_clear = delete_outlier_data(data_all,symbols)
    best_cv_df = get_best_cv_for_miss_symbols(missing_symbols)
    # best_model_missing =  train_model(data_all,missing_symbols,best_cv_df)
    # merged_model = save_model(best_model_tuning,best_model_missing)
    pred_y_label_all = predict_all_sym(merged_model,data_clear)
    pred_all = simplify_output(pred_y_label_all)
    return pred_all

In [23]:
symbols = {'all': ['EURJPY']}

In [24]:
pred_y_label_all = get_signal(symbols)

EURJPY


Eval_CV:   0%|          | 0/5 [00:00<?, ?it/s]

EURJPY000
                      model              log_loss            accuracy
0  average_by_rf_xgb_mlp000    -0.454155299720914  0.8289473684210527
1  average_by_rf_xgb_mlp000  -0.46108373625358334   0.819078947368421
2  average_by_rf_xgb_mlp000   -0.4687722926451905  0.7894736842105263
3  average_by_rf_xgb_mlp000  -0.43896570302871096   0.805921052631579
4  average_by_rf_xgb_mlp000  -0.45264471690567254  0.8283828382838284


KeyError: 'EURCHF'

{'EURJPY000': [RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=300),
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=1.0, device=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=0.5, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=0.05, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=8, max_leaves=None,
                min_child_weight=5, missing=nan, monotone_constraints=None,
                multi_strategy=None, n_estimators=100, n_jobs=None,
                num_parallel_tree=None, random_state=2023, ...),
  MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=(120, 80, 40),
                max_iter=80, solver='sgd')]}

In [ ]:
# file_name = 'best_model_tuning.pkl'
# # Lưu từ điển vào tệp pickle
# with open(file_name, 'wb') as file:
#     pickle.dump(pred_y_label_all, file)